# Questions to Answer 
1. Is the data homogenous in each column?

Yes

2. How do you anticipate this data will be used by data analysts and scientists downstream?

This could be used to search for patterns in storm development and damage in order to help understand trends and minimize future damages.

3. Does your answer to the last question give you an indication of how you can store the data for optimal querying speed and storage file compression?

Generally yes, I have a good idea of how I would like to store and compress the data based on the access requirements. 

4. What cleaning steps do you need to perform to make your dataset ready for consumption?

The data has been cleaned prior to the point where i pick it up, but I may perform certain joins based on testing.

5. What wrangling steps do you need to perform to enrich your dataset with additional information?

No wrangling or enrichment required.

In [ ]:
#Process Used:

Joined the three tables(details,locations,fatalities) on the primary key column Event_ID using q/kdb+ to determine 

event_id is unique for each row in detail (53187 rows in latest file)

every fatality has an event_id match in detail (799 rows in latest file)
  those have repeated values, so the distinct subset is less (434 rows in latest file)
some fatalities have an event_id match in location (149 rows in latest file)
  those have repeated values, so the distinct subset is less (89 rows in latest file)

every location has an event_id match in detail (44760 rows in latest file)
  those have repeated values, so the distinct subset is less (25965 rows in latest file)
some locations have an event_id match in fatality (299 rows in latest file)
  those have repeated values, so the distinct subset is less (89 rows in latest file)
    
q script: "clean data.q"


In [1]:
"""q script: "clean data.q"
\c 10 3000

detfiles:asc hsym each `$'system "ls | grep details"
locfiles:asc hsym each `$'system "ls | grep locat"
fatfiles:asc hsym each `$'system "ls | grep fatal"

det: (,/) {(51#"*";enlist ",") 0:x} each detfiles
loc: (,/) {(11#"*";enlist ",") 0:x} each locfiles
fat: (,/) {(11#"*";enlist ",") 0:x} each fatfiles


update BEGIN_DATE:(BEGIN_YEARMONTH,'BEGIN_DAY) from `det where not 1=count each BEGIN_DAY
update BEGIN_DATE:(BEGIN_YEARMONTH,'("0",'BEGIN_DAY)) from `det where 1=count each BEGIN_DAY
update END_DATE:(END_YEARMONTH,'END_DAY) from `det where not 1=count each END_DAY
update END_DATE:(END_YEARMONTH,'("0",'END_DAY)) from `det where 1=count each END_DAY
update BEGIN_DATETIME:"Z"$((BEGIN_DATE),'(" ",'-8#'BEGIN_DATE_TIME)) from `det
update END_DATETIME:"Z"$((END_DATE),'(" ",'-8#'END_DATE_TIME)) from `det

update "D"$10#'FATALITY_DATE from `fat

update "M"$YEARMONTH from loc

update "I"$EPISODE_ID,"I"$EVENT_ID,`$STATE,"I"$INJURIES_DIRECT,"I"$INJURIES_INDIRECT,"I"$DEATHS_DIRECT,"I"$DEATHS_INDIRECT,"F"$MAGNITUDE,`$MAGNITUDE_TYPE,"I"$CATEGORY,`$TOR_F_SCALE,"F"$TOR_LENGTH,"F"$TOR_WIDTH,"F"$BEGIN_LAT,"F"$BEGIN_LON,"F"$END_LAT,"F"$END_LON,`$DATA_SOURCE from `det
update "I"$EPISODE_ID,"I"$EVENT_ID,"I"$LOCATION_INDEX,"F"$RANGE,`$AZIMUTH,`$LOCATION,"F"$LATITUDE,"F"$LONGITUDE,"I"$LAT2,"I"$LON2 from `loc
update "I"$FATALITY_ID,"I"$EVENT_ID,`$FATALITY_TYPE,"I"$FATALITY_AGE,`$FATALITY_SEX from `fat

fat:fat lj (`EVENT_ID xkey select EPISODE_ID,EVENT_ID from det)

sdet: select BEGIN_DATETIME,END_DATETIME,EPISODE_ID,EVENT_ID,STATE,EVENT_TYPE,CZ_NAME,INJ_DIR:INJURIES_DIRECT,INJ_IND:INJURIES_INDIRECT,DEAD_DIR:DEATHS_DIRECT,DEAD_IND:DEATHS_INDIRECT,
  DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE from det
sloc: select EPISODE_ID,EVENT_ID,LOCATION,LATITUDE,LONGITUDE from loc
sfat: select FATALITY_DATE,FATALITY_ID,EVENT_ID,EPISODE_ID,FATALITY_TYPE,FATALITY_AGE,FATALITY_SEX,FATALITY_LOCATION from fat

allmatch:(sdet ij `EPISODE_ID xkey sloc) ij `EPISODE_ID xkey sfat
allfull:(sdet lj `EPISODE_ID xkey sloc) lj `EPISODE_ID xkey sfat

compmatch:?[`allmatch;not,'enlist each @[(null;0=count');"C"=exec t from meta allmatch],'cols allmatch;0b;()]
compmatch:`BEGIN_DATETIME`EPISODE_ID`EVENT_ID xdesc compmatch

save `:compmatch.csv

randeps:desc 10?count select i from compmatch where i = (min;i) fby EPISODE_ID
randfull:select from compmatch where i in randeps
save `:randfull.csv
"""

'q script: "clean data.q"\n\\c 10 3000\n\ndetfiles:asc hsym each `$\'system "ls | grep details"\nlocfiles:asc hsym each `$\'system "ls | grep locat"\nfatfiles:asc hsym each `$\'system "ls | grep fatal"\n\ndet: (,/) {(51#"*";enlist ",") 0:x} each detfiles\nloc: (,/) {(11#"*";enlist ",") 0:x} each locfiles\nfat: (,/) {(11#"*";enlist ",") 0:x} each fatfiles\n\n\nupdate BEGIN_DATE:(BEGIN_YEARMONTH,\'BEGIN_DAY) from `det where not 1=count each BEGIN_DAY\nupdate BEGIN_DATE:(BEGIN_YEARMONTH,\'("0",\'BEGIN_DAY)) from `det where 1=count each BEGIN_DAY\nupdate END_DATE:(END_YEARMONTH,\'END_DAY) from `det where not 1=count each END_DAY\nupdate END_DATE:(END_YEARMONTH,\'("0",\'END_DAY)) from `det where 1=count each END_DAY\nupdate BEGIN_DATETIME:"Z"$((BEGIN_DATE),\'(" ",\'-8#\'BEGIN_DATE_TIME)) from `det\nupdate END_DATETIME:"Z"$((END_DATE),\'(" ",\'-8#\'END_DATE_TIME)) from `det\n\nupdate "D"$10#\'FATALITY_DATE from `fat\n\nupdate "M"$YEARMONTH from loc\n\nupdate "I"$EPISODE_ID,"I"$EVENT_ID,`$STA

### ERD Diagram in Slide Deck


### Google Slide Deck: Exploratory_Data_Analysis.pdf